In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install faiss-gpu-cu11

In [3]:
import os, json
import torch
from PIL import Image
import numpy as np
from tqdm import tqdm
import faiss
from transformers import AutoModel, AutoImageProcessor

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:::", device)

Device::: cuda


In [5]:
# Đường dẫn trên Google Drive
DATA_PATH = "/content/drive/MyDrive/dinov2/"
IMAGE_DIR = os.path.join(DATA_PATH, "collection/images")
INDEX_PATH = os.path.join(DATA_PATH, "core/oxford_index.bin")
JSON_PATH = os.path.join(DATA_PATH, "core/oxford_embeddings.json")
os.makedirs(os.path.dirname(INDEX_PATH), exist_ok=True)

In [6]:
# Load model và processor
dinov2_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base', use_fast=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

In [7]:
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dinov2_model(**inputs)
        embedding = outputs.pooler_output  # (1, 768)

    return embedding.squeeze(0).cpu().numpy()  # (768,)

In [8]:
files = sorted([
    os.path.join(IMAGE_DIR, f)
    for f in os.listdir(IMAGE_DIR)
    if f.endswith(".jpg")
])

In [9]:
# FAISS index khởi tạo với 768 chiều
index = faiss.IndexFlatL2(768)
all_embeddings = {}

# Tạo embedding & add vào FAISS index
for img_path in tqdm(files):
    emb = extract_embedding(img_path)
    index.add(emb.reshape(1, -1))
    all_embeddings[os.path.basename(img_path)] = emb.tolist()

# Lưu FAISS index
faiss.write_index(index, INDEX_PATH)


100%|██████████| 5063/5063 [11:23<00:00,  7.41it/s]


In [10]:
# Lưu mapping file -> embedding
with open(JSON_PATH, "w") as f:
    json.dump(all_embeddings, f)

print("Done saving FAISS index and embeddings!")

Done saving FAISS index and embeddings!
